In [ ]:
import setup  # notebooks/setup.py
setup.init()

In [ ]:
from mypermit import permit_client

permit = permit_client

In [ ]:
await permit.api.users.list()

In [ ]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [ ]:
for u in User.objects.all():
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}"
    }
    response = await permit.api.users.sync(user_data)
    print(response)

In [ ]:
await permit.check("1", "read", "document")

In [ ]:
resources = [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [ ]:
from permit import PermitApiError

async def get_or_create_resource(resource_data:dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    return _resource

In [ ]:
for resource_data in resources:
    response = await get_or_create_resource(resource_data)
    print(response)

In [ ]:
# readonly_permissions = [
#     "document:read",
#     "movie_disovery:search",
#     "movie_disovery:detail"
# ]

In [ ]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }, 
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }
]

In [ ]:
from permit import PermitApiError

async def get_or_create_role(role_data:dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    return _resource

In [ ]:
# await permit.api.roles.delete("viewer")

In [ ]:
for role_data in roles:
    response = await get_or_create_role(role_data)
    print(response)

In [ ]:
user = User.objects.all().first()

In [ ]:
user.id

In [ ]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "admin",
    "tenant": "default"
})
response

In [ ]:
await permit.check(f"{user.id}", "delete", "document")

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver 
from ai.supervisors import get_supervisor
import uuid 

checkpointer = InMemorySaver()
supervisor = get_supervisor(model="llama3.1:8b", checkpointer=checkpointer)

In [ ]:
doc_id = 24
user_id = user.id
user_id

In [ ]:

config = {"configurable": {"user_id": user_id, "thread_id": uuid.uuid4()}}

response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": "what are my recent documents?"}
        ]
    },
    config
)

In [ ]:
response['messages'][-1].content

In [ ]:
response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": f"Delete any random document by id"}
        ]
    },
    config
)
response['messages'][-1].content